## OrthoDB Data

Scripts for the download and basic preparation of fasta files containing ortholog sequences.

The source is OrthoDB's api.

In [14]:
import requests, time
import sys

from tqdm import tqdm
import pandas as pd
import json
from datasets.orthodb_preparation.data_utils import *

BASE = "https://data.orthodb.org/v12/download/odb_data_dump"
DATASET_ROOT= Path("../fasta_files")
OUT = Path("dumps")
OUT.mkdir(parents=True, exist_ok=True)

#### Download of resources and raw files
Get index files from api

In [2]:
files = [
    "odb12v2_levels.tab.gz",
    "odb12v2_level2species.tab.gz",
    "odb12v2_OG2genes.tab.gz",
    "odb12v2_genes.tab.gz",
    "odb12v2_OGs.tab.gz"
]
for fn in files:
    url = f"{BASE}/{fn}"
    dst = OUT / fn
    if dst.exists() and dst.stat().st_size > 0:
        continue
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total=int(r.headers.get("content-length", 0))
        with open(dst, "wb") as f, tqdm(total=total, unit="B", unit_scale=True, desc=fn) as pbar:
            for chunk in r.iter_content(chunk_size=1<<20):
                if chunk: f.write(chunk); pbar.update(len(chunk))

odb12v2_levels.tab.gz: 100%|██████████| 16.0k/16.0k [00:00<?, ?B/s]
odb12v2_level2species.tab.gz: 100%|██████████| 247k/247k [00:00<00:00, 2.34MB/s]
odb12v2_OG2genes.tab.gz: 100%|██████████| 4.81G/4.81G [04:04<00:00, 19.7MB/s]
odb12v2_genes.tab.gz: 100%|██████████| 4.84G/4.84G [03:39<00:00, 22.0MB/s]
odb12v2_OGs.tab.gz: 100%|██████████| 134M/134M [00:05<00:00, 24.8MB/s] 


Get orthologous groups of a clade

In [8]:
CLADE = "Mammalia"  # Vertebrate/Mammalia/Eukaryota/…

levels = pd.read_csv(
    OUT/"odb12v2_levels.tab.gz",
    sep="\t", header=None, compression="gzip",
    names=["level_taxid","name","nr_genes","nr_ogs","nr_species"]  # <-- 5 colonne
)

level_id = levels.loc[levels["name"]==CLADE, "level_taxid"].iloc[0]
print("Clade:", CLADE, "→ level_taxid:", level_id)

ogs_df = pd.read_csv(
    OUT/"odb12v2_OGs.tab.gz",
    sep="\t", header=None, compression="gzip",
    names=["og_id","level_taxid","og_name"]
)

ogs_in_clade = ogs_df.loc[ogs_df["level_taxid"]==level_id, "og_id"].drop_duplicates()
print("OG trovati:", len(ogs_in_clade))

Path("ogs.txt").write_text("\n".join(ogs_in_clade.tolist()))

Clade: Mammalia → level_taxid: 40674
OG trovati: 42608


591195

Download every fasta file in chosen clade

In [10]:
API = "https://data.orthodb.org/v12/fasta"
raw_dir = DATASET_ROOT / "orthodb_v12/raw"; raw_dir.mkdir(parents=True, exist_ok=True)

# if not none, max number of files downloaded
max_download_count = 500

def download_cds_for_og(og_id):
    url = f"{API}?id={og_id}&seqtype=cds"
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    (raw_dir/f"{og_id}.cds.fasta").write_text(r.text, encoding="utf-8")

ogs = [og for og in Path("ogs.txt").read_text().splitlines() if og]
total_iter = min(max_download_count, len(ogs)) if max_download_count else len(ogs)


for i, og in enumerate(tqdm(ogs[:total_iter], total=min(total_iter, max_download_count), desc="Scarico OG", unit="og", file=sys.stdout), 1):
    if not og: continue
    out = raw_dir/f"{og}.cds.fasta"
    if out.exists() and out.stat().st_size>0:
        continue
    try:
        download_cds_for_og(og)
    except Exception as e:
        tqdm.write(f"Failed {i}: {e}")
    if i % 50 == 0:
        time.sleep(1)

    tqdm.write(f"Downloaded {i} {og}")


Downloaded 1 100009at40674                         
Downloaded 2 100011at40674                                 
Downloaded 3 100068at40674                                 
Downloaded 4 100125at40674                                 
Downloaded 5 100128at40674                                 
Downloaded 6 100187at40674                                 
Downloaded 7 100244at40674                                 
Downloaded 8 100287at40674                                 
Downloaded 9 100314at40674                                 
Downloaded 10 100670at40674                                
Downloaded 11 100672at40674                                 
Downloaded 12 100717at40674                                 
Downloaded 13 100729at40674                                 
Downloaded 14 100751at40674                                 
Downloaded 15 100791at40674                                 
Downloaded 16 100882at40674                                 
Downloaded 17 100969at40674               

#### Basic cleaning
Counting unique characters

In [15]:
fasta_dir = DATASET_ROOT / "orthodb_v12/raw"
patterns = ("*.fasta", "*.fa", "*.faa", "*.fna")

symbols = Counter()

for fasta_path in iter_fasta_files(fasta_dir, patterns=patterns):
    records = read_fasta(fasta_path)
    for _, seq in records:
        symbols.update(seq.upper())

# Remove empty key if present
if "" in symbols:
    del symbols[""]

print("Unique symbols found in sequences:")
print(" ".join(sorted(symbols.keys())))

print("\nCount per symbol:")

for sym, count in symbols.most_common():
    print(f"{sym!r}: {count}")

Unique symbols found in sequences:
A C G K M N R S T W Y

Count per symbol:
'G': 43172693
'C': 42636975
'A': 42447123
'T': 38339684
'N': 13590
'Y': 532
'R': 487
'M': 113
'S': 94
'K': 60
'W': 52


Resolve ambiguous bases, using the distribution of the certain ones

In [17]:
input_dir = DATASET_ROOT / "orthodb_v12/raw"
output_dir = DATASET_ROOT / "orthodb_v12/no_ambig_imp"
output_dir.mkdir(parents=True, exist_ok=True)

base_counts = {b : symbols[b] for b in "ACGT"}
total = sum(base_counts.values())
base_probs = {b : base_counts[b] / total for b in "ACGT"}

Path("global_base_probs.json").write_text(
    json.dumps(base_probs, indent=2)
)

print(f"Base counts: {base_counts}")
print(f"Total base counts: {total}")
print(f"Base probabilities: {base_probs}")

for in_path in tqdm(
        list(iter_fasta_files(
            input_dir,
            patterns=("*.fasta", "*.fa", "*.fna")
        )),
        desc="Resolving ambiguous bases",
        unit="file"
):
    out_path = output_dir / in_path.name

    records = read_fasta(in_path)
    sequences = [seq for _, seq in records]

    new_records = []
    for header, seq in records:
        new_seq = resolve_ambiguous_sequence(seq, base_probs)
        new_records.append((header, new_seq))

    write_fasta(out_path, new_records, width=60)

print("Done.")

Base counts: {'A': 42447123, 'C': 42636975, 'G': 43172693, 'T': 38339684}
Total base counts: 166596475
Base probabilities: {'A': 0.2547900428265364, 'C': 0.2559296347656816, 'G': 0.2591452970418492, 'T': 0.23013502536593286}


Resolving ambiguous bases: 100%|██████████| 500/500 [00:11<00:00, 43.47file/s]

Done.


Check for duplicate sequence

In [18]:
input_dir = DATASET_ROOT / "orthodb_v12/no_ambig_imp"

for path in iter_fasta_files(input_dir, patterns=("*.fasta", "*.fa", "*.fna", "*.faa", "*.cds.fasta")):
    records = read_fasta(path)
    duplicates = find_duplicate_sequences(records)

    print(f"\n=== {path.name} ===")
    if duplicates:
        print(f"Found {len(duplicates)} duplicated sequences:")
        for seq, count in duplicates.items():
            print(f" - occurrences: {count}")
    else:
        print("No duplicated sequences found.")


=== 100009at40674.cds.fasta ===
No duplicated sequences found.

=== 100011at40674.cds.fasta ===
Found 57 duplicated sequences:
 - occurrences: 2
 - occurrences: 4
 - occurrences: 4
 - occurrences: 2
 - occurrences: 3
 - occurrences: 2
 - occurrences: 3
 - occurrences: 3
 - occurrences: 3
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 4
 - occurrences: 14
 - occurrences: 3
 - occurrences: 3
 - occurrences: 2
 - occurrences: 2
 - occurrences: 4
 - occurrences: 8
 - occurrences: 2
 - occurrences: 3
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 4
 - occurrences: 5
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 6
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 2
 - occurrences: 3
 - occurrences: 3
 - occurrences: 4
 - occurrences: 2
 - occurrences: 2
 - occu

Remove duplicate sequences

In [19]:
input_dir = DATASET_ROOT / "orthodb_v12/no_ambig_imp"
output_dir = DATASET_ROOT / "orthodb_v12/unique_no_ambig_imp"
output_dir.mkdir(parents=True, exist_ok=True)

for in_path in iter_fasta_files(input_dir, patterns=("*.fasta", "*.fa", "*.faa", "*.fna", "*.cds.fasta")):
    records = read_fasta(in_path)
    out_records = make_unique_records(records)
    out_path = output_dir / in_path.name

    write_fasta(out_path, out_records)
    print(
        f"{in_path.name}: {len(records)} sequences → "
        f"{len(out_records)} unique sequences saved."
    )

print("\nDone.")

100009at40674.cds.fasta: 6 sequences → 6 unique sequences saved.
100011at40674.cds.fasta: 609 sequences → 503 unique sequences saved.
100068at40674.cds.fasta: 9 sequences → 9 unique sequences saved.
100125at40674.cds.fasta: 7 sequences → 7 unique sequences saved.
100128at40674.cds.fasta: 3 sequences → 3 unique sequences saved.
100187at40674.cds.fasta: 7 sequences → 7 unique sequences saved.
100244at40674.cds.fasta: 22 sequences → 22 unique sequences saved.
100287at40674.cds.fasta: 394 sequences → 330 unique sequences saved.
100314at40674.cds.fasta: 397 sequences → 391 unique sequences saved.
100670at40674.cds.fasta: 20 sequences → 20 unique sequences saved.
100672at40674.cds.fasta: 409 sequences → 386 unique sequences saved.
100717at40674.cds.fasta: 402 sequences → 398 unique sequences saved.
100729at40674.cds.fasta: 143 sequences → 115 unique sequences saved.
100751at40674.cds.fasta: 132 sequences → 132 unique sequences saved.
100791at40674.cds.fasta: 314 sequences → 301 unique sequen

Optionally cut sequences longer than K to a random length included in [K-H, K]

In [20]:
IN_DIR = DATASET_ROOT / "orthodb_v12/unique_no_ambig_imp"
OUT_DIR = DATASET_ROOT / "orthodb_v12/unique_no_ambig_imp_cut"
OUT_DIR.mkdir(parents=True, exist_ok=True)

K = 300   # threshold: sequences longer than K will be cut
H = 100   # interval: new length will be in [K-H, K]


def process_file(fin: Path):
    records = read_fasta(fin)
    if not records:
        return

    new_records = []
    lengths_before = []
    lengths_after = []

    for h, s in records:
        orig_len = len(s)
        new_len = random_cut_length(orig_len, K=K, H=H)
        new_s = s[:new_len]
        new_records.append((h, new_s))
        lengths_before.append(orig_len)
        lengths_after.append(new_len)

    fout = OUT_DIR / fin.name
    write_fasta(fout, new_records, width=WRAP_DEFAULT)

    print(
        f"{fin.name} -> {fout} | nseq={len(records)} | "
        f"len_before=[{min(lengths_before)}–{max(lengths_before)}] | "
        f"len_after=[{min(lengths_after)}–{max(lengths_after)}]"
    )


for fin in iter_fasta_files(IN_DIR, patterns=("*.fasta", "*.fa", "*.fna", "*.cds.fasta")):
    process_file(fin)

print("Done.")

100009at40674.cds.fasta -> ..\datasets\fasta_files\orthodb_v12\unique_no_ambig_imp_cut\100009at40674.cds.fasta | nseq=6 | len_before=[276–453] | len_after=[253–298]
100011at40674.cds.fasta -> ..\datasets\fasta_files\orthodb_v12\unique_no_ambig_imp_cut\100011at40674.cds.fasta | nseq=503 | len_before=[192–864] | len_after=[192–300]
100068at40674.cds.fasta -> ..\datasets\fasta_files\orthodb_v12\unique_no_ambig_imp_cut\100068at40674.cds.fasta | nseq=9 | len_before=[303–987] | len_after=[219–286]
100125at40674.cds.fasta -> ..\datasets\fasta_files\orthodb_v12\unique_no_ambig_imp_cut\100125at40674.cds.fasta | nseq=7 | len_before=[465–807] | len_after=[204–299]
100128at40674.cds.fasta -> ..\datasets\fasta_files\orthodb_v12\unique_no_ambig_imp_cut\100128at40674.cds.fasta | nseq=3 | len_before=[312–312] | len_after=[206–263]
100187at40674.cds.fasta -> ..\datasets\fasta_files\orthodb_v12\unique_no_ambig_imp_cut\100187at40674.cds.fasta | nseq=7 | len_before=[360–480] | len_after=[200–299]
100244at